In [2]:
import os

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12777: async pool tids: 140372239427328, 140372237326080, 140372235224832
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12777: async pool tids: 140372233123584, 140372228925184
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12777: async pool tids: 140372224726784, 140372216334080
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12777: async pool tids: 140372207941376, 140372191160064
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12777: async pool tids: 140372174378752, 140372172277504


In [1]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:
#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, AMfLD3s2EGy4SEAjYUveoSr96jrj335CbtQ33h8vnFyf, 0>


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:
# ##################################################################

In [ ]:
#  IMPORT DATA INTO GRAPH USING CSV

# l_inp="gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-0.003-bi-node_headers.txt"
# l_dir="gs://katana-demo-datasets/csv-datasets/ldbc"

l_inp="gs://farrell-data-bucket/D1_DateTest.txt"
l_dir="gs://farrell-data-bucket/"

# l_inp="gs://farrell-data-bucket/40_sf-0.003-bi-node_headers.txt"
# l_dir="gs://farrell-data-bucket/"


import_data.csv(
   my_graph,
      #
   input_node_path    = l_inp,
   input_dir          = l_dir,
      #
   data_delimiter     = "|",
   schema_delimiter   = "|",
   files_have_headers = False
   )

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:
result = my_graph.query("""

   MATCH (n)
   RETURN n
   
   """)

print(result)


In [ ]:
# ##################################################################

In [ ]:
#  This was checking to see if birthday was a string
#  No result

result = my_graph.query("""

   MATCH (n)
   WHERE n.birthday ='2017-04-04'
   RETURN properties(n)
   
   """)

print(result)


In [ ]:
#  This is a check to see if birthday is a datetime
#  No result

result = my_graph.query("""

   MATCH (n)
   WHERE n.birthday = 2017-04-04
   RETURN properties(n)
   
   """)

print(result)


In [ ]:
#  This should force birthday to a string but fails
#  Errored

result = my_graph.query("""

   MATCH (n)
   WHERE toString(n.birthday) ='2017-04-04' 
   RETURN properties(n)
   
   """)

print(result)


In [ ]:
#  Birthday can not be parsed
#  Errored

result = my_graph.query("""

   MATCH (n)
   WHERE n.birthday STARTS WITH '2017' 
   RETURN properties(n)
   
   """)

print(result)


In [ ]:
# ##################################################################

In [ ]:

#   Jira 3910,  done/closed

result = my_graph.query("""

   MATCH (n) 
   RETURN count(distinct labels(n))
   
   """)

print(result)
